In [1]:
import pandas as pd
import folium
import numpy as np
import matplotlib as mpl
from folium.plugins import TimestampedGeoJson
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

pd.set_option('display.precision', 2)


data_path_feb = "../Data/01_Courses_usagers/2023_02/"
data_path_jun = "../Data/01_Courses_usagers/2023_06/"

render_map_path = "render_map/"

In [2]:
stations = pd.read_csv(data_path_feb + "All_data_courses.csv", sep=";").loc[:, ['from_stat', 'Latitude_start', 'longitude_start']].drop_duplicates()
stations = stations.rename(columns={"from_stat": "nom", "Latitude_start": "lat", "longitude_start": "long"})

print(stations)

                                               nom    lat  long
0                        Jean Moulin - Paul Doumer  48.86  2.46
1                          Lucie Aubrac - Franklin  48.86  2.44
2             Docteurs Déjérine - Pte de Montreuil  48.85  2.41
3                          Jules Vallès - Charonne  48.85  2.38
5        Gare de l'Est - Place du 11 Novembre 1918  48.88  2.36
...                                            ...    ...   ...
2249988                   Quai Panhard et Levassor  48.83  2.38
2282809                  Ney - Porte de Montmartre  48.90  2.34
2655446                      Jean Macé - Faidherbe  48.85  2.38
2790915                        Centenaire - Sorins  48.86  2.43
2972450              Stalingrad - Désiré Chevalier  48.86  2.44

[1471 rows x 3 columns]


In [3]:
avg_loc = stations[['lat', 'long']].mean().to_list()
map_paris = folium.Map(locaion=avg_loc, zoom_start=15)

for station in stations.itertuples():
    marker = folium.CircleMarker(location=(float(station.lat), float(station.long)), tooltip=station.nom, radius=2, weight=2)
    marker.add_to(map_paris)
    
sw = stations[['lat', 'long']].min().values.tolist()
ne = stations[['lat', 'long']].max().values.tolist()
map_paris.fit_bounds([sw, ne])
map_paris

## Trajets

In [4]:
courses_02 = pd.read_csv(data_path_feb + "All_data_courses.csv", sep=";").loc[:, ['id_velo', 'date_depart', 'date_arrive', 'from_stat', 'to_stat']]

# get column date and hour
datage = courses_02.loc[:, 'date_depart'].str.split(' ', expand=True)
jours = datage.iloc[:, 0]
heures = datage.iloc[:, 1]

# Add date and hour
courses_02.insert(courses_02.shape[1], 'jour', jours.str.split('-', expand=True).iloc[:, 2].values)
courses_02.insert(courses_02.shape[1], 'heure', heures.str.split(':', expand=True).iloc[:, 0].values)

# Rinse & repeat
# courses_06 = pd.read_csv(data_path_jun + "All_data_courses.csv").loc[:, ['id_velo', 'date_depart', 'date_arrive', 'from_stat', 'to_stat']]

In [23]:
trajets_02 = courses_02.groupby(['from_stat', 'to_stat', 'heure']).size().reset_index(name='count')
trajets_02 = trajets_02.loc[trajets_02['count'] > 2, :]
trajets_02['count'] /= max(trajets_02['count'])
trajets_02['count'] = round(trajets_02['count'], 2)
print(trajets_02.shape)
print(trajets_02)
# trajets_06 = courses_06.groupby(['from_stat', 'to_stat']).size().reset_index(name='count')
# trajets_06['count'] /= max(trajets_06['count'])

(225343, 4)
                       from_stat               to_stat heure  count
32       18 juin 1940 - Buzenval      Chanzy - Sud-Est    09   0.03
34       18 juin 1940 - Buzenval      Chanzy - Sud-Est    13   0.03
37       18 juin 1940 - Buzenval  Colmar - Victor Hugo    07   0.06
38       18 juin 1940 - Buzenval  Colmar - Victor Hugo    08   0.07
39       18 juin 1940 - Buzenval  Colmar - Victor Hugo    09   0.05
...                          ...                   ...   ...    ...
1922385          Île de la Jatte       Île de la Jatte    16   0.05
1922386          Île de la Jatte       Île de la Jatte    17   0.03
1922387          Île de la Jatte       Île de la Jatte    18   0.03
1922388          Île de la Jatte       Île de la Jatte    19   0.03
1922390          Île de la Jatte       Île de la Jatte    21   0.03

[225343 rows x 4 columns]


In [24]:
heures = trajets_02.loc[:, ['heure']].drop_duplicates().values
liste_heures = []
for i in heures:
    liste_heures.append(i[0])

print(sorted(liste_heures))

['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']


In [13]:
courses_02

,id_velo,date_depart,date_arrive,from_stat,to_stat,jour,heure
0,2041,2023-02-01 07:09:23,2023-02-01 07:13:00,Jean Moulin - Paul Doumer,Lucie Aubrac - Franklin,01,07
1,2041,2023-02-01 11:51:49,2023-02-01 12:01:34,Lucie Aubrac - Franklin,Docteurs Déjérine - Pte de Montreuil,01,11
2,2041,2023-02-01 12:09:27,2023-02-01 12:21:51,Docteurs Déjérine - Pte de Montreuil,Jules Vallès - Charonne,01,12
3,2041,2023-02-01 12:27:40,2023-02-01 12:54:56,Jules Vallès - Charonne,Borrégo - Gambetta,01,12
4,2041,2023-02-01 14:02:18,2023-02-01 14:18:36,Jules Vallès - Charonne,Gare de l'Est - Place du 11 Novembre 1918,01,14
...,...,...,...,...,...,...,...
3049909,68021,2023-02-28 17:38:46,2023-02-28 18:04:25,Paul Bodin - Clichy,Mathis - Flandre,28,17
3049910,68021,2023-02-28 18:04:58,2023-02-28 18:21:05,Mathis - Flandre,Hector Malot - Daumesnil,28,18
3049911,68021,2023-02-28 18:35:23,2023-02-28 18:51:40,Hector Malot - Daumesnil,Pelleport - Belgrand,28,18
3049912,68021,2023-02-28 18:56:27,2023-02-28 19:07:52,Pelleport - Belgrand,Gambetta - Michelet,28,18


### Color gradient 

In [14]:
def colorFader(c1,c2,mix=0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    c1=np.array(mpl.colors.to_rgb(c1))
    c2=np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

c1='#1f77b4' #blue
c2='#ab2025' #red

In [26]:
map_paris_02 = folium.Map(locaion=avg_loc, zoom_start=15, prefer_canvas=True)
map_paris_02.fit_bounds([sw, ne])

features = []
station_before = ""
i = 0
for index, row in tqdm(trajets_02.iterrows(), total=trajets_02.shape[0] // 4):
    if i >= trajets_02.shape[0] // 4:
        break
    else:
        i += 1
    # for index, row in trajets_02.iterrows():
    if row['from_stat'] != station_before:
        coord_ft = stations.loc[stations['nom'] == row['from_stat'], ['lat', 'long']].values[0].tolist()
        station_before = row['from_stat']
    coord_tt = stations.loc[stations['nom'] == row['to_stat'], ['lat', 'long']].values[0].tolist()
    coords = [coord_ft, coord_tt]
    
    feature = {
        'type': 'Feature',
        'geometry': {
            'type': 'LineString',
            'coordinates': coords
        },
        'properties': {
            'times': [row['heure']] * len(coords),
            'fillColor': colorFader(c1, c2, row['count'])
        }
    }
    features.append(feature)
    
print("Markers added")
TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': features},
    period='P1D',
    duration='P1D',
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='HH',
    time_slider_drag_update=True
).add_to(map_paris_02)

print("Done")
map_paris_02.save("map2.html")
print("Saved")

  0%|          | 0/56335 [00:00<?, ?it/s]

Markers added
Done
Saved
